In [6]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# Load env
load_dotenv()

DB_HOST = os.environ.get("DB_HOST")
DB_NAME = os.environ.get("DB_NAME")
DB_USER = os.environ.get("DB_USER")
DB_PASS = os.environ.get("DB_PASS")

# Create engine
engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}:3306/{DB_NAME}")

# Test connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1"))
    print(result.fetchone())


(1,)


In [ ]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
import pandas as pd
import random
from faker import Faker
import numpy as np

# Load env
load_dotenv()
DB_HOST = os.environ.get("DB_HOST")
DB_NAME = os.environ.get("DB_NAME")
DB_USER = os.environ.get("DB_USER")
DB_PASS = os.environ.get("DB_PASS")

# SQLAlchemy engine
engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}:3306/{DB_NAME}")

# ---------- Step 1: Generate Synthetic Data ----------
fake = Faker()
Faker.seed(42)
random.seed(42)

# Users
users = []
for i in range(10000):
    users.append({
        "user_id": i+1,
        "name": fake.name(),
        "age": random.randint(18,65),
        "gender": random.choice(["Male","Female","Other"]),
        "location": fake.city(),
        "signup_date": fake.date_between(start_date="-5y", end_date="today")
    })
users_df = pd.DataFrame(users)

# Advertisers
advertisers = []
for i in range(200):
    advertisers.append({
        "advertiser_id": i+1,
        "company_name": fake.company(),
        "industry": random.choice(["Tech","Fashion","Food","Travel","Finance"]),
        "budget": random.randint(10000,100000)
    })
advertisers_df = pd.DataFrame(advertisers)

# Ads
ads = []
for i in range(500):
    ads.append({
        "ad_id": i+1,
        "advertiser_id": random.randint(1,200),
        "category": random.choice(["Tech","Fashion","Food","Travel","Finance"]),
        "ad_text": fake.catch_phrase(),
        "budget": random.randint(500,5000)
    })
ads_df = pd.DataFrame(ads)

# Impressions
impressions = []
for i in range(100000):
    impressions.append({
        "impression_id": i+1,
        "user_id": random.randint(1,10000),
        "ad_id": random.randint(1,500),
        "timestamp": fake.date_time_between(start_date="-30d", end_date="now"),
        "device": random.choice(["Mobile","Desktop","Tablet"]),
        "clicked": np.random.choice([0,1], p=[0.9,0.1])
    })
impressions_df = pd.DataFrame(impressions)

# User Features
user_features = []
for i in range(1,10001):
    user_features.append({
        "user_id": i,
        "avg_session_length": round(random.uniform(5,120),2),
        "avg_ads_seen": random.randint(5,100),
        "preferred_device": random.choice(["Mobile","Desktop","Tablet"]),
        "active_days_last_30": random.randint(1,30),
        "avg_click_rate": round(random.uniform(0,0.2),3)
    })
user_features_df = pd.DataFrame(user_features)

# ---------- Step 2: Create Tables ----------
with engine.begin() as conn:  # begin() auto-commits
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS users (
            user_id INT PRIMARY KEY,
            name VARCHAR(100),
            age INT,
            gender VARCHAR(10),
            location VARCHAR(100),
            signup_date DATE
        )
    """))
    
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS advertisers (
            advertiser_id INT PRIMARY KEY,
            company_name VARCHAR(100),
            industry VARCHAR(50),
            budget INT
        )
    """))
    
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS ads (
            ad_id INT PRIMARY KEY,
            advertiser_id INT,
            category VARCHAR(50),
            ad_text VARCHAR(255),
            budget INT,
            FOREIGN KEY (advertiser_id) REFERENCES advertisers(advertiser_id)
        )
    """))
    
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS ad_impressions (
            impression_id INT PRIMARY KEY,
            user_id INT,
            ad_id INT,
            timestamp DATETIME,
            device VARCHAR(20),
            clicked TINYINT,
            FOREIGN KEY (user_id) REFERENCES users(user_id),
            FOREIGN KEY (ad_id) REFERENCES ads(ad_id)
        )
    """))
    
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS user_features (
            user_id INT PRIMARY KEY,
            avg_session_length FLOAT,
            avg_ads_seen INT,
            preferred_device VARCHAR(20),
            active_days_last_30 INT,
            avg_click_rate FLOAT,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        )
    """))

users_df.to_sql("users", engine, if_exists="append", index=False)
advertisers_df.to_sql("advertisers", engine, if_exists="append", index=False)
ads_df.to_sql("ads", engine, if_exists="append", index=False)
impressions_df.to_sql("ad_impressions", engine, if_exists="append", index=False)
user_features_df.to_sql("user_features", engine, if_exists="append", index=False)

print("All tables created and data inserted successfully!")


All tables created and data inserted successfully!
